<a href="https://colab.research.google.com/github/Anyaoma/My-projects/blob/main/Live_Trading_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os
import json
import math
import pandas as pd
import numpy as np
import pickle

In [3]:
pip install oandapyV20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for oandapyV20: filename=oandapyV20-0.7.2-py3-none-any.whl size=69779 sha256=97af446e5faf27e16913a40a741977226e033af02fb876db4e33f5a8976d2725
  Stored in directory: /root/.cache/pip/wheels/8e/69/ab/a6da84a85b9bf3b5a98ca15c6c52b7854f32b10c70fe0531a1
Successfully built oandapyV20


In [4]:
import oandapyV20
import oandapyV20.endpoints
from oandapyV20 import API
import oandapyV20.endpoints.accounts as v20Accounts
import oandapyV20.endpoints.instruments as v20Instruments
import oandapyV20.endpoints.pricing as v20Pricing
import oandapyV20.endpoints.orders as v20Orders
import oandapyV20.endpoints.positions as v20Positions
import oandapyV20.contrib.requests as v20Requests
import oandapyV20.endpoints.trades as v20Trades
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from oandapyV20.exceptions import StreamTerminated

In [9]:
model = pickle.load(open('model_AB_classification.pkl', 'rb'))

In [10]:
train_stats = pd.read_csv('train_stats.csv')
train_stats.index = train_stats['Unnamed: 0']

In [11]:
algorithm = {
    'model': model,
    'mu': train_stats.iloc[:,1],
    'std': train_stats.iloc[:,2]
}

In [12]:
class MLStrategy(OandaTrader1):
    def __init__(self,  symbol,time_length,file_name, algorithm):
        super(MLStrategy, self).__init__(file_name)
        self.model = algorithm['model']
        self.symbol = symbol
        self.mean = algorithm['mu']
        self.std = algorithm['std']
        self.units = 100
        self.position = 0
        self.time_length = '5s'
        self.window = 2
        self.lags = 6
        self.min_length = self.lags + self.window +1
        self.features = ['return', 'volatility', 'momentum', 'sma', 'min', 'max']
        self.ordertype = 'MARKET'
        self.raw_data = pd.DataFrame()

    def prepare_features(self):
        self.data['return'] = np.log(self.data['close']/self.data['close'].shift(1))
        self.data['volatility'] = self.data['return'].rolling(self.window).std()
        self.data['momentum'] = np.sign(self.data['return'].rolling(self.window).mean())
        self.data['sma'] = self.data['close'].rolling(self.window).mean()
        self.data['min'] = self.data['close'].rolling(self.window).min()
        self.data['max'] = self.data['close'].rolling(self.window).max()

        self.data.dropna(inplace=True)
        self.cols = []
        for f in self.features:
            for lag in  range(1, self.lags + 1):
                col = f'{f}_lag_{lag}'
                self.data[col] = self.data[f].shift(lag)
                self.cols.append(col)
        self.data.dropna(inplace=True)
        self.data -= self.mean
        self.data /= self.std
        print(self.data.head(5))

    def place_trade(self,time, bid, ask, symbol):
        self.raw_data = pd.concat([self.raw_data,pd.DataFrame({'bid':float(bid),'ask':float(ask)}, index = [pd.Timestamp(time).tz_localize(None)])])
        self.data = self.raw_data.resample(self.time_length, label='right').last().ffill().iloc[:-1]
        if len(self.data) > self.min_length:
            self.min_length += 1
            self.data['close'] = self.data.mean(axis=1)
            self.data.drop(columns=['bid','ask'],inplace=True)
            #print(self.data.head())
            self.prepare_features()
            print('preparing features')
            features = self.data[self.cols].iloc[-1].values.reshape(1, -1)
            signal = self.model.predict(features)[0]
            print(f'NEW SIGNAL: {signal}',end ='\r')
            if self.position in [0, -1] and signal == 1:
                print('*** GOING LONG ***')
                self.PlaceOrder(symbol, self.ordertype, units=(1-self.position)*self.units)
                self.position = 1
            elif self.position in [0, 1] and signal == -1:
                print('*** GOING SHORT ***')
                self.PlaceOrder(self.symbol, self.ordertype, units=-(1+self.position)*self.units)
                self.position = -1

    def stream_data(self, symbol, stop):
        params = {"instruments": symbol }
        req = v20Pricing.PricingStream(accountID=self.credentials['AccountID'], params=params)
        results = self.API("StreamData",req)

        maxrecs = stop
        done = False
        while not done:
            try:
                for ticks in results:
                    new_data = {}
                    #data.append(ticks)
                    new_data['time'] = ticks['time']
                    new_data['bid'] = ticks['bids'][0]['price']
                    new_data['ask'] = ticks['asks'][0]['price']

                    self.place_trade(time = new_data['time'], bid = float(new_data['bid']), ask = float(new_data['ask']), symbol=symbol)

                    #print(new_data['time'])
                    #print(new_data['bid'])
                    #print(new_data['ask'])
                    maxrecs -= 1
                    if maxrecs == 0:
                        req.terminate()
            except KeyError as err:
                del new_data['time']
                continue
            except StreamTerminated as e:
                done = True
                break
            else:
                self._place_trade(time = new_data['time'], bid = float(new_data['bid']), ask = float(new_data['ask']), units=units )


In [13]:
class OandaTrader1:

    def __init__(self, file_name=None):
        self.credentials = self.GetCredentials(file_name)
        self.Broker = self.Login()
        self.Markets = self.GetMarkets()

        #Load credentials from file

    def GetCredentials(self, file_name):
        credentials = {}
        if os.path.exists(file_name):
            try:
                filehandle = open(file_name,'r')
                credentials['AccountID'] = filehandle.readline().strip()
                credentials['BearerToken'] = filehandle.readline().strip()
                filehandle.close()
            except:
                print(f'Please check your {file_name} missing or malformed data')
                sys.exit(1)
        else:
            print(f'you must save your OANDA account credentials to: {file_name}')
            sys.exit(1)

        return credentials

    #Login to the exchange

    def Login(self):
        try:
            oanda = API(access_token = self.credentials['BearerToken'])
        except Exception as err:
            print('Error:',str(err))
            sys.exit(1)

        return oanda

    #Handle all OANDA requests with retry loop

    def API(self, function, req):
        retry = 0
        RetryLimit = 3

        done = False
        while not done:
            try:
                result = self.Broker.request(req)
            except Exception as err:
                if retry > RetryLimit:
                    print(str(err))
                    sys.exit(1)
                else:
                    em = str(err)
                    print(f"{function}:Retrying({retry+1}), {em}")
            else:
                done = True
            retry+=1
        return result

    #Retrieve tradeable instruments

    def GetMarkets(self):
        req = v20Accounts.AccountInstruments(accountID=self.credentials['AccountID'])
        results = self.API('GetMarket', req)
        return results

    #Retrive ticker information

    def GetTicker(self,symbol):
        params = {'instruments':symbol}
        req = v20Pricing.PricingInfo(accountID=self.credentials['AccountID'],params=params)
        ticker = self.API('GetTicker', req)

        #Build the forex pair dictionary
        ForexPair = {}
        ForexPair['Time'] = pd.Timestamp(ticker['time'])
        ForexPair['Ask'] = round(float(ticker['prices'][0]['asks'][0]['price']),5)
        ForexPair['Bid'] = round(float(ticker['prices'][0]['bids'][0]['price']),5)
        ForexPair['Spread'] = round(ForexPair['Ask']-ForexPair['Bid'],5)
        return ForexPair

     # Read the order book

    def GetOrderBook(self, symbol):
        req = v20Instruments.InstrumentsOrderBook(instrument=symbol,params={})
        results = self.API("GetOrderBook",req)
        return results['orderBook']['buckets']

     # Read the currnt open positions
    def GetOpenPositions(self, symbol):
        symbol = symbol.replace('/','_')
        params={"instruments":symbol }
        req=v20Trades.TradesList(self.credentials['AccountID'],params=params)
        results=self.API("GetOpenPosiions",req)
        return results['trades']

    # Place a trade/open a position

    def PlaceOrder(self, pair, orderType, units, price=None):
        pair = pair.replace('/','_')
        orderType = orderType.upper()
        order = {}
        if orderType == 'LIMIT':
            order['price'] = str(price)
            order['timeInForce'] = 'GTC'
        order['units'] = str(units)
        order['instrument'] = pair
        order['type'] = orderType
        order['positionFill'] = 'DEFAULT'
        params = {}
        params['order'] = order
        res=v20Orders.OrderCreate(self.credentials['AccountID'],data=params)
        results=self.API("OrderCreate",res)
        return results

    #Close a trade or position

    def CloseOrder(self, pair, units, price=None):
        params = {}
        if str(units).upper()!='ALL':
            #amount is STR, need float for abs()
            units = float(units)
            if float(units)>=0:
                params['longUnits']=str(math.floor(abs(units)))
            else:
                params['shortUnits']=str(math.floor(abs(units)))
        else:
            params['longUnits'] = 'ALL'
        res=v20Positions.PositionClose(self.credentials['AccountID'],instrument=pair,data=params)
        results=self.API("PositionClose",res)


        return results

    #Get historial data
    def get_history(self,instrument, time_frame, from_, end ):
        instrument, granularity = instrument, time_frame
        _from = from_

        params = {"from": _from,
                  "to": end,
              "granularity": granularity,
              }

        all_candle = []
        output_directory = "/tmp"
        file_path = os.path.join(output_directory, "{}.{}".format(instrument, granularity))
        os.makedirs(output_directory, exist_ok=True)

        for req in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = self.API('GetHistory',req)
            all_candle.extend(response.get('candles'))
            #all_candles.extend(r.response.get('candles'))

        with open(file_path, "w") as OUT:
            json.dump(all_candle, OUT, indent=2)

        with open(file_path, "r") as file:
            data = json.load(file)

        candles = data if isinstance(data, list) else data['candles']
        df = pd.DataFrame(candles)

        columns = ['volume','time','open','high','low','close']
        open_price = []
        high_price = []
        low_price = []
        close_price = []
        for price in df['mid']:
            open_price.append(price['o'])
            high_price.append(price['h'])
            low_price.append(price['l'])
            close_price.append(price['c'])

        df['open'] = open_price
        df['high'] = high_price
        df['low'] = low_price
        df['close'] = close_price

        df = df[columns]

        return df

    #Get account summary
    def get_account_summary(self):
        req= v20Accounts.AccountSummary(accountID=self.credentials['AccountID'])
        request = self.API('GetAccountSummary', req)
        print(json.dumps(request, indent=2))
